In [26]:
import numpy as np
import pandas as pd
import requests as rq
import math
from scipy import stats
import xlsxwriter

In [27]:
stocks = pd.read_csv('/Users/hashimkhan/Desktop/algo_trading_final/sp_500_stocks.csv')
from secret_case import API_TOKEN

In [28]:
#Function sources from: https://stackoverflow.com/questions/312443/how-do-i-split-a-list-into-equally-sized-chunks
def chunks (lst, n):
    """Yield succisve n-sized chunks form lst"""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

In [44]:
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
dataframes = []

for symbol_string in symbol_strings:
    batch_api_url = "https://yfapi.net/v6/finance/quote"
    
    querystring = {"symbols": symbol_string}
    
    headers = {
        'x-api-key': API_TOKEN
    }

    data = rq.get(batch_api_url, headers=headers, params=querystring).json()

    rows = []
    for result in data.get('quoteResponse', {}).get('result', []):
        symbol = result.get('symbol', np.nan)
        
        year_high_percent = float(result.get('fiftyTwoWeekLowChangePercent', np.nan))
        year_low_percent = float(result.get('fiftyTwoWeekHighChangePercent', np.nan))

        year_price_return = ((year_high_percent + year_low_percent) / 2)

        regular_market_price = float(result.get('regularMarketPrice', np.nan))

        rows.append([symbol, regular_market_price, year_price_return, np.nan])

    dataframe = pd.DataFrame(rows, columns=my_columns)
    dataframes.append(dataframe)

final_dataframe = pd.concat(dataframes, ignore_index=True)
final_dataframe

Ticker   Price  One-Year Price Return  Number of Shares to Buy
0        A  110.50              -0.148367                      NaN
1      AAL   12.61              -0.128348                      NaN
2      AAP   54.82              -0.356030                      NaN
3     AAPL  170.43               0.116156                      NaN
4     ABBV  153.13               0.040090                      NaN
..     ...     ...                    ...                      ...
499    YUM  121.98               0.012371                      NaN
500    ZBH  110.76              -0.089179                      NaN
501   ZBRA  225.91              -0.171387                      NaN
502   ZION   33.17               0.203902                      NaN
503    ZTS  174.68               0.151424                      NaN

[504 rows x 4 columns]

In [45]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.head()

index Ticker   Price  One-Year Price Return  Number of Shares to Buy
0    345   NVDA  424.68               1.386180                      NaN
1    194    FTI   21.29               0.810315                      NaN
2    390    RCL   91.96               0.652478                      NaN
3    198     GE  110.89               0.617819                      NaN
4     84    CCL   13.98               0.501571                      NaN

In [46]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')

    try:
        float(portfolio_size)
    except ValueError:
        print("That is not a number! \nPlease try again:")
        portfolio_size = input('Enter the size of your portfolio:')

portfolio_input()
print(portfolio_size)

1000000


In [47]:
position_size = float(portfolio_size) /len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Price'])

final_dataframe.head()

index Ticker   Price  One-Year Price Return  Number of Shares to Buy
0    345   NVDA  424.68               1.386180                     47.0
1    194    FTI   21.29               0.810315                    939.0
2    390    RCL   91.96               0.652478                    217.0
3    198     GE  110.89               0.617819                    180.0
4     84    CCL   13.98               0.501571                   1430.0

## Below Contains a More Realistic Momentum Strategy

In [33]:
hqm_columns = [
    'Ticker',
    'Price',
    'One-Year Price Return',
    'One-Year Return Percentile',
    '200-Day Price Return',
    '200-Day Return Percentile',
    '50-Day Price Return',
    '50-Day Return Percentile',
]

In [48]:
dataframes1 = []
rows1 = []

for symbol_string in symbol_strings:
    batch_api_url = "https://yfapi.net/v6/finance/quote"
    
    querystring = {"symbols": symbol_string}
    
    headers = {
        'x-api-key': API_TOKEN
    }

    data1 = rq.get(batch_api_url, headers=headers, params=querystring).json()

    for result in data1.get('quoteResponse', {}).get('result', []):
        
        symbol = result.get('symbol', np.nan)
       
        year_high_percent = float(result.get('fiftyTwoWeekLowChangePercent', np.nan))
        year_low_percent = float(result.get('fiftyTwoWeekHighChangePercent', np.nan))

        year_price_return = ((year_high_percent + year_low_percent) / 2)

        regular_market_price = float(result.get('regularMarketPrice', np.nan))
        day200_return = float(result.get('twoHundredDayAverageChangePercent', np.nan))
        day50_return = float(result.get('fiftyDayAverageChangePercent', np.nan))

        rows1.append([symbol, regular_market_price, year_price_return, 'n/a', day200_return, 'n/a', day50_return, 'n/a'])

    dataframe = pd.DataFrame(rows1, columns=hqm_columns)
    dataframes1.append(dataframe)

final_dataframe = pd.concat(dataframes1, ignore_index=True)

In [49]:
final_dataframe.head()

Ticker   Price  One-Year Price Return One-Year Return Percentile  \
0      A  110.50              -0.148367                        n/a   
1    AAL   12.61              -0.128348                        n/a   
2    AAP   54.82              -0.356030                        n/a   
3   AAPL  170.43               0.116156                        n/a   
4   ABBV  153.13               0.040090                        n/a   

   200-Day Price Return 200-Day Return Percentile  50-Day Price Return  \
0             -0.167640                       n/a            -0.081727   
1             -0.165346                       n/a            -0.159468   
2             -0.485734                       n/a            -0.178987   
3              0.027022                       n/a            -0.064560   
4              0.022247                       n/a             0.026917   

  50-Day Return Percentile  
0                      n/a  
1                      n/a  
2                      n/a  
3                      n/a  
4                      n/a